In [ ]:
pip install torch transformers datasets rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using ca

In [ ]:
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')
article = dataset['train']['article']
summary = dataset['train']['highlights']
ids = dataset['train']['id']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Define data chunking function
def data_chunks(dataset, chunk_size=40000):
    article = dataset['train']['article']
    summary = dataset['train']['highlights']
    ids = dataset['train']['id']
    chunked_data = []
    for i in range(0, len(article), chunk_size):
        chunk = {
            'id': ids[i:i+chunk_size],
            'article': article[i:i+chunk_size],
            'summary': summary[i:i+chunk_size]
        }
        chunked_data.append(chunk)
        # Save each chunk as a CSV file
        chunk_df = pd.DataFrame(chunk)
        chunk_df.to_csv(f'cnn_chunk_{i//chunk_size}.csv', index=False)
    return chunked_data

chunked_data = data_chunks(dataset)

In [ ]:
# Define preprocessing functions
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

def preprocess_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return text

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

def remove_punctuation(text):
    exclude = set(string.punctuation)
    return ''.join(char for char in text if char not in exclude)

def clean_fields(df):
    df['article'] = df['article'].apply(lambda x: remove_url(x))
    df['summary'] = df['summary'].apply(lambda x: remove_url(x))

    df['article'] = df['article'].apply(lambda x: remove_html(x))
    df['summary'] = df['summary'].apply(lambda x: remove_html(x))

    # df['article'] = df['article'].apply(lambda x: preprocess_text(x))
    # df['summary'] = df['summary'].apply(lambda x: preprocess_text(x))

    # df['article'] = df['article'].apply(lambda x: remove_punctuation(x))
    # df['summary'] = df['summary'].apply(lambda x: remove_punctuation(x))
    return df

In [ ]:
# Apply preprocessing to each chunk and save
for i, chunk in enumerate(chunked_data):
    chunk_df = pd.DataFrame(chunk)
    clean_chunk_df = clean_fields(chunk_df)
    clean_chunk_df.to_csv(f'clean_cnn_chunk_{i}.csv', index=False)

In [ ]:
# Load and print the first few rows of the cleaned chunk
clean_chunk_df = pd.read_csv('clean_cnn_chunk_2.csv')
print(clean_chunk_df.head())

#print the min-avg-max-median of the length of the articles
article_length = clean_chunk_df['article'].apply(lambda x: len(str(x).split()))
print(f"Min: {article_length.min()}")
print(f"Max: {article_length.max()}")
print(f"Avg: {article_length.mean()}")
print(f"Median: {article_length.median()}")
print(f"Standard Deviation: {article_length.std()}")
print(f"Variance: {article_length.var()}")

                                         id  \
0  b1abfc993a55b7b27001e8a692e5ac6f20825d05   
1  006dd02051a6d228fa3841ab555fced21fccdf1e   
2  bc31ad108b7dbc7f8489c4ea2426d3340a912333   
3  a5227658a7f7f58286c9175a8cd471c5e946050a   
4  a9d8a27ef232db714a45bf44c468859ad57422c9   

                                             article  \
0  (CNN) -- Flamboyant carnival musician Michel M...   
1  Kenyan counterterrorism sources are looking at...   
2  Unmanned aerial vehicles aren't generally thou...   
3  Washington (CNN) -- One of two photojournalist...   
4  (CNN)A nurse who was quarantined against her w...   

                                             summary  
0  NEW: Streets fill with jubilant Michel Martell...  
1  A Norwegian citizen of Somali descent is inves...  
2  The UAE launched the Drones for Good Award las...  
3  Footage of U.S. helicopter attack on two Reute...  
4  Nurse Kaci Hickox: "I don't plan on sticking t...  
Min: 32
Max: 2058
Avg: 740.51405
Median: 679.0
Sta

In [ ]:
#print the min-avg-max-median of the length of the summaries
summary_length = clean_chunk_df['summary'].apply(lambda x: len(str(x).split()))
print(f"Min: {summary_length.min()}")
print(f"Max: {summary_length.max()}")
print(f"Avg: {summary_length.mean()}")
print(f"Median: {summary_length.median()}")
print(f"Standard Deviation: {summary_length.std()}")
print(f"Variance: {summary_length.var()}")

Min: 4
Max: 1296
Avg: 43.142975
Median: 41.0
Standard Deviation: 19.10731169954017
Variance: 365.0893603833846


In [ ]:
pip install bert-extractive-summarizer pandas

In [ ]:
import pandas as pd
from summarizer import Summarizer
from rouge_score import rouge_scorer

df = clean_chunk_df

# Initialize the BERT summarizer
model = Summarizer()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Function to summarize an article
def summarize_article(article):
    return model(article)

# Function to compute ROUGE scores
def compute_rouge_scores(reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

In [ ]:
generated_summaries = []
rouge_scores_list = []

# Iterate through the DataFrame
for index, row in df.iterrows():
    article = row['article']
    reference_summary = row['summary']

    # Generate summary
    generated_summary = summarize_article(article)
    generated_summaries.append(generated_summary)

    # Compute ROUGE scores
    rouge_scores = compute_rouge_scores(reference_summary, generated_summary)
    rouge_scores_list.append(rouge_scores)

df['generated_summary'] = generated_summaries
df['rouge_scores'] = rouge_scores_list

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
df['generated_summary'] = pd.Series(generated_summaries)
df['rouge_scores'] = pd.Series(rouge_scores_list)

print(df[['id', 'article', 'summary', 'generated_summary', 'rouge_scores']])

df.to_excel('results.xlsx', index=True)

                                             id  \
0      b1abfc993a55b7b27001e8a692e5ac6f20825d05   
1      006dd02051a6d228fa3841ab555fced21fccdf1e   
2      bc31ad108b7dbc7f8489c4ea2426d3340a912333   
3      a5227658a7f7f58286c9175a8cd471c5e946050a   
4      a9d8a27ef232db714a45bf44c468859ad57422c9   
...                                         ...   
39995  6bbba9c734b4fa4dbc1d48c5a94b49a1ec8bdd73   
39996  395b9196646dae7dc6763c13810f51aff94237ba   
39997  35cbd5e633e6c6619039aea8af78cdb842c81685   
39998  b2058a13e781f5d2935225281407d13fde12034f   
39999  2845fe6220c452d1b74830216bacbb1363c8dc03   

                                                 article  \
0      (CNN) -- Flamboyant carnival musician Michel M...   
1      Kenyan counterterrorism sources are looking at...   
2      Unmanned aerial vehicles aren't generally thou...   
3      Washington (CNN) -- One of two photojournalist...   
4      (CNN)A nurse who was quarantined against her w...   
...                        